##import the Wikidata integrator

In [4]:
%%capture
!pip install wikidataintegrator
from wikidataintegrator import wdi_core, wdi_login, wdi_config

## Get all Wikidata properties

In [10]:
sparql = "https://data.lexbib.org/query/sparql"

query = """
SELECT DISTINCT ?property ?propertyLabel ?propertyDescription ?propType WHERE {
   ?property wikibase:directClaim ?p ;
             wikibase:propertyType ?propType ;
             schema:description ?propertyDescription ;
             rdfs:label ?propertyLabel .
   FILTER (lang(?propertyLabel) = 'en')
   FILTER (lang(?propertyDescription) = 'en')}
"""
propertiesSparql = wdi_core.WDItemEngine.execute_sparql_query(query, endpoint=sparql, as_dataframe=True)
propertiesSparql

datatype_map = {}
for datatype in wdi_config.property_value_types.keys():
    datatype_map[wdi_config.property_value_types[datatype]] = datatype
propertiesSparql['datatype']= ""
for index, row in propertiesSparql.iterrows():
    row["datatype"] = datatype_map[row["propType"]] 
propertiesSparql 

,property,propType,propertyDescription,propertyLabel,datatype
0,http://data.lexbib.org/entity/P88,http://wikiba.se/ontology#String,written representation of a term candidate ext...,corpus keyword,string
1,http://data.lexbib.org/entity/P90,http://wikiba.se/ontology#WikibaseItem,Lexical Conceptual Resource type,LCR type,wikibase-item
2,http://data.lexbib.org/entity/P48,http://wikiba.se/ontology#WikibaseItem,reverse property: Author (P12),is author of,wikibase-item
3,http://data.lexbib.org/entity/P55,http://wikiba.se/ontology#WikibaseItem,ms:distribution,physical distribution,wikibase-item
4,http://data.lexbib.org/entity/P56,http://wikiba.se/ontology#WikibaseItem,object language (used for automatic indexation),object language candidate,wikibase-item
5,http://data.lexbib.org/entity/P57,http://wikiba.se/ontology#Url,equivalent MetaShare property,MetaShare Property,url
6,http://data.lexbib.org/entity/P58,http://wikiba.se/ontology#Url,equivalent MetaShare class,MetaShare Ontology Class,url
7,http://data.lexbib.org/entity/P59,http://wikiba.se/ontology#WikibaseItem,ms:mediaPart,media part,wikibase-item
8,http://data.lexbib.org/entity/P60,http://wikiba.se/ontology#WikibaseItem,"inverse property of ""has distribution""",distribution of,wikibase-item
9,http://data.lexbib.org/entity/P63,http://wikiba.se/ontology#WikibaseItem,ms:isContinuedBy,is continued by,wikibase-item


## Login to Wikibase

In [11]:
from getpass import getpass
import pprint

# https://data.lexbib.org
wikibase = "https://lexbibclone.wiki.opencura.com/"
api = wikibase+"w/api.php"
sparql = wikibase+"query/sparql"
entityUri = wikibase.replace("https:", "http:")+"entity/"

WBUSER = getpass(prompt="username:")  
WBPASS = getpass(prompt='Enter your password: ')  
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=api)

username:········
Enter your password: ········


#### Function to create properties

In [14]:
def createProperty(login=login, wdprop=None, label="", description="", property_datatype=""):
    if wdprop== None:
        s = []
    else:
        s = [wdi_core.WDUrl(wdprop, prop_nr="P1")]
    localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
    item = localEntityEngine(data=s)
    item.set_label(label)
    item.set_description(description)
    print(item.write(login, entity_type="property", property_datatype=property_datatype))

#create mapping to wikidata property

#createProperty(login, label="property in wikidata", description="The same property in Wikidata", property_datatype="url")
  

#### Create Wikidata properties

In [16]:
for row in propertiesSparql.itertuples():
    print(datatype_map[row.propType])
    print(row)
    createProperty(login=login, label=row.propertyLabel, description=row.propertyDescription, property_datatype=datatype_map[row.propType])

string
Pandas(Index=0, property='http://data.lexbib.org/entity/P88', propType='http://wikiba.se/ontology#String', propertyDescription='written representation of a term candidate extracted from corpora', propertyLabel='corpus keyword', datatype='string')
P2
wikibase-item
Pandas(Index=1, property='http://data.lexbib.org/entity/P90', propType='http://wikiba.se/ontology#WikibaseItem', propertyDescription='Lexical Conceptual Resource type', propertyLabel='LCR type', datatype='wikibase-item')
P3
wikibase-item
Pandas(Index=2, property='http://data.lexbib.org/entity/P48', propType='http://wikiba.se/ontology#WikibaseItem', propertyDescription='reverse property: Author (P12)', propertyLabel='is author of', datatype='wikibase-item')
P4
wikibase-item
Pandas(Index=3, property='http://data.lexbib.org/entity/P55', propType='http://wikiba.se/ontology#WikibaseItem', propertyDescription='ms:distribution', propertyLabel='physical distribution', datatype='wikibase-item')
P5
wikibase-item
Pandas(Index=4, p